This notebook is used for SI 650 Information Retrieval class. You should implement retrieval functions and report corresponding results in your submission on Canvas. 


In [126]:
# install metapy, it may take several minutes.
!pip install metapy
import metapy

In [127]:
# Reading Data
!wget -nc https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt
!wget -N https://meta-toolkit.org/data/2016-11-10/cranfield.tar.gz
!tar xf cranfield.tar.gz
!wget -N http://www-personal.umich.edu/~shiyansi/cacm.tar.gz
!tar xf cacm.tar.gz

File ‘lemur-stopwords.txt’ already there; not retrieving.

--2018-10-18 21:38:32--  https://meta-toolkit.org/data/2016-11-10/cranfield.tar.gz
Resolving meta-toolkit.org (meta-toolkit.org)... 50.116.41.177, 2600:3c02::f03c:91ff:feae:b777
Connecting to meta-toolkit.org (meta-toolkit.org)|50.116.41.177|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘cranfield.tar.gz’ not modified on server. Omitting download.

--2018-10-18 21:38:36--  http://www-personal.umich.edu/~shiyansi/cacm.tar.gz
Resolving www-personal.umich.edu (www-personal.umich.edu)... 141.211.243.103
Connecting to www-personal.umich.edu (www-personal.umich.edu)|141.211.243.103|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘cacm.tar.gz’ not modified on server. Omitting download.



In [ ]:
# Setting cranfield dataset
with open('cranfield/tutorial.toml', 'w') as f:
    f.write('type = "line-corpus"\n')
    f.write('store-full-text = true\n')

config = """prefix = "." # tells MeTA where to search for datasets

dataset = "cranfield" # a subfolder under the prefix directory
corpus = "tutorial.toml" # a configuration file for the corpus specifying its format & additional args

index = "cranfield-idx" # subfolder of the current working directory to place index files

query-judgements = "cranfield/cranfield-qrels.txt" # file containing the relevance judgments for this dataset

stop-words = "lemur-stopwords.txt"

[[analyzers]]
method = "ngram-word"
ngram = 1
filter = "default-unigram-chain"
"""
with open('cranfield-config.toml', 'w') as f:
    f.write(config)

In [ ]:
# Setting cacm dataset
with open('cacm/tutorial.toml', 'w') as f:
    f.write('type = "line-corpus"\n')
    f.write('store-full-text = true\n')

config = """prefix = "." # tells MeTA where to search for datasets

dataset = "cacm" # a subfolder under the prefix directory
corpus = "tutorial.toml" # a configuration file for the corpus specifying its format & additional args

index = "cacm-idx" # subfolder of the current working directory to place index files

query-judgements = "cacm/cacm-qrels.txt" # file containing the relevance judgments for this dataset

stop-words = "lemur-stopwords.txt"

[[analyzers]]
method = "ngram-word"
ngram = 1
filter = "default-unigram-chain"
"""
with open('cacm-config.toml', 'w') as f:
    f.write(config)

In [ ]:
# Make sure you have installed metapy package and downloaded the data before running the following code

In [ ]:
# Build the index for dataset.
inv_idx_cran = metapy.index.make_inverted_index('cranfield-config.toml')
inv_idx_cacm = metapy.index.make_inverted_index('cacm-config.toml')

#** 3 Define New Retrieval Function**

**Please write your own retrieval function in the cell below**

In [ ]:
import math
class  NewRF (metapy.index.RankingFunction):                                            
                                                                       
    def __init__(self, k1=2.2, b=0.9, k3=1000, c1=1, c2=6):
      self.k1 = k1
      self.b = b
      self.k3 = k3
      self.c1 = c1
      self.c2 = c2
      super(NewRF, self).__init__() 

        #recommended parameters from the literature: (k1=1.25, b=0.90, k3=1000)
#         self.param = some_param

#      super(NewRF, self).__init__()                                        
                                                                                 
    def score_one(self, sd):
        """
        You need to override this function to return a score for a single term.
        
        You may want to call some of the following variables when implementing your retrieval function:
        sd.avg_dl: average document length of the collection
        sd.num_docs: total number of documents in the index
        sd.total_terms: total number of terms in the index
        sd.query_length: the total length of the current query (sum of all term weights)
        sd.query_term_weight: query term count (or weight in case of feedback)
        sd.doc_count: number of documents that a term t_id appears in
        sd.corpus_term_count: number of times a term t_id appears in the collection
        sd.doc_term_count: number of times the term appears in the current document
        sd.doc_size: total number of terms in the current document
        sd.doc_unique_terms: number of unique terms in the current document
        """
        #Write your answer here collection
        #(k1=1.25, b=0.90, k3=1000)
        
        
        from math import log as log
        from math import e as e
        k1 = self.k1
        b = self.b
        k3 = self.k3
        c1 = self.c1
        c2 = self.c2
        
        # Proposed Algorithm BM25-CTF::
        # BM25-CTF model = sum(bidf(w)) * (((k1 + 1) * btf(w,d))/(k1 *K(d) + btf(w,d)) * ((k3 +1) * btf(w,q)/(k3 +btf(w,q)))
            # w = an index term (or word)
            # d = doc as a collection of index terms
           
            
        # K(d) = doc-length normalization factor from BM25 = 
        
        # factor_one = bidf(w) = ictf(w) × pidf(w) × idf(w)
        # M = total num terms in collection = log(num of docs/num docs term appears) = log(D/df(w)) = log(sd.num_docs/sd.doc_count)
        # ctf(w) = collection term freq. = sd.corpus_term_count
        # ictf(w) = inverse collec. term freq factor = log(M/ctf(w)) 
        
        # Part 1 of the Equation: bidf_w
        m = float(sd.total_terms)

        ctf_w = float(sd.corpus_term_count) #num docs with a word
        D = float(sd.num_docs)
        df_w = float(sd.doc_count)
    
        #ictf_w
        ictf_w = log(m / ctf_w, 10)
      
        # pidf_w = log((df_w_hat/ctf_w_hat) + 1)
        df_w_hat = D * (1 - e**(-(ctf_w/D)))
        ctf_w_hat = -D * log(1 - (df_w / D))
        pidf_w = log((df_w_hat/ctf_w_hat) + 1, 10)
        
        # idf_w = -log(1 - e(-(ctf_w/|D|)))
        idf_w = -log(1 - e**(-(ctf_w / D)))
        
        #factor_one = bidf_w
        factor_one = ictf_w * pidf_w * idf_w
        
        
        #Part 2 of the equation:
        # Part 2 numerator = (k1+1) * btf_wd
            # btf_wd = C(d) * (tf_wd/tf_wd_hat)
#         tf_wd = sd.doc_term_count 
# #         ctf_wprime = 1
#         tf_wd_hat = 1 + (1 * (D - sd.doc_unique_terms) )
        
#         tf_wpd =
#         tf_wpd_hat = 
#         cd = D / (tf_wpd/tf_wpd_hat)
#         btf_wd = cd * (tf_wd/tf_wd_hat)
#         kd = 
        
#         factor_two_numerator = (k1+1) * btf_wd
#         factor_two_denominator = k1 * kd * btf_wd
#         factor_two = factor_two_numerator/factor_two_denominator
        
        c_td = float(sd.doc_term_count)
        d_size = float(sd.doc_size)
        a_dl = float(sd.avg_dl)
        
        # create ddc, part_a=dd (document density), factor_one_a=ddc (document density)
        part_a = ((c_td/D)/df_w)
        factor_one_a = c1 * (log(part_a,10) + c2)
   
        # Part 2 of the equation
        factor_two_numerator = (k1 + 1) * c_td
        factor_two_denominator = (k1 * (1 - b + b *(d_size/a_dl))) + c_td
        factor_two = factor_two_numerator/factor_two_denominator
        
        
        #Part 3 of the equation
        factor_three_numerator = (k3 + 1) * sd.query_term_weight
        factor_three_denominator = (k3 + sd.query_term_weight)
        factor_three = factor_three_numerator / factor_three_denominator 
        
        result = (factor_one + factor_one_a) * factor_two * factor_three
# 
        
        
        return result

In [ ]:
ranker = NewRF()

In [133]:
ev = metapy.index.IREval('cranfield-config.toml')
num_results = 30

precision_list = []
with open('cranfield/cranfield-queries.txt') as query_file:
    for query_num, line in enumerate(query_file):
        query = metapy.index.Document()
        query.content(line.strip())
        results = ranker.score(inv_idx_cran, query, num_results)                            
        avg_p = ev.avg_p(results, query_num + 1, num_results)
        precision_list.append(ev.precision(results,query_num+1,num_results))

print "MAP", ev.map()
print "Precision@30", sum(precision_list) / len(precision_list)


MAP 0.290234156138
Precision@30 0.124888888889


In [134]:
ev2 = metapy.index.IREval('cacm-config.toml')
num_results = 30

precision_list = []
with open('cacm/cacm-queries.txt') as query_file:
    for query_num, line in enumerate(query_file):
        query = metapy.index.Document()
        query.content(line.strip())
        results = ranker.score(inv_idx_cacm, query, num_results)                            
        avg_p = ev2.avg_p(results, query_num + 1, num_results)
        precision_list.append(ev2.precision(results, query_num+1, num_results))

print "MAP", ev2.map()
print "Precision@30", sum(precision_list) / len(precision_list)


MAP 0.238704532083
Precision@30 0.1703125


# Testing Search Results for a Single Query

In [ ]:
query = metapy.index.Document()
query.content("ibm")
top_docs = ranker.score(inv_idx_cacm, query, num_results=5)

In [136]:
for num, (d_id, _) in enumerate(top_docs):
    content = inv_idx_cacm.metadata(d_id).get('content')
    print("{}. {}...\n".format(num + 1, content))

1. ibm 704 code nundrums...

2. character scanning on the ibm 7070...

3. counting ones on the ibm 7090...

4. statistical programs for the ibm 650 part i a collection is given of brief descriptions of statistical programs now in use in university computing centers which have ibm 650...

5. realizing boolean connectives on the ibm 1620...



**Please submit your code for  NewRF class to canvas. We need your code to verify your results.**

**Reasons for choosing the models I chose**:

For this assignment, for the article, I searched the U of M library search engine. My query was BM25 and among the top articles (or the 4th article) that was retrieved in the Articles section was the article, *BM25-CTF: Improving TF and IDF factors in BM25 by using collection term frequencies*. In the article, it states that BM25-CTF imporves BM25 by 17.6% for the TREC dataset and 7.6% averaged across all collections included in the test. This is the main reason for trying this algorithm. Looking within the article, it seems that they found other algorithms that beat the Okapi BM25 algorithm, however, the BM25-CTF seems to perform better than those.  The only downfall of using this algorithm is that when the collection documents are not as long, the BM25-CTF will not perform as well. 

To further improve the BM25-CTF algorithm, I added parameters suggested in the article, *Word Document Density and Relevance Scoring*. The parameters they suggested were ddc  and dd(which is within ddc). This parameter helps to capture the quantity of word repetitiveness and is added to idf in BM25. Adding this parameter helped raise the MAP score for the Cranfield dataset - beating the BM25 score, but not the CACM dataset. 

The recommended values for k1, b, and k3 were as follows: (k1=1.25, b=0.90, k3=1000). Since the k3 values do not matter as much for this equation, I changed the b and k1 values to make my model run better. I found when k1=1.2, b=0.9, k3=100 it improved my MAP scores for the Cranfield and CACM data. For the c1 and c2 values, I looked at the values retrieval performance for average precision on TREC and the best performance seemed to be at c1=1, c2=6.

**Articles used**:

Jimenez, Sergio, Silviu-Petru Cucerzan, Fabio A. Gonzalez, Alexander Gelbukh, and George Duenas, *BM25-CTF: Improving TF and IDF factors in BM25 by using collection term frequencies*. Journal of Intelligent & Fuzzy Systems 34, 2018, pp.2887–2899.

Frans, Martin and J. Scott McCarley, *Word Document Density and Relevance Scoring*. In *Proceedings of the 23rd ACM SIGIR*, 2000, pp.345-347.